```
QUERYNAME = CholesterolMSMS;
DEFINE PR = 'C[41..49] H[30..200] O[2] N[1]' WITH DBR = (4.5,10.5), CHG = +1;
DEFINE FR ='C[27] H[45]' WITH DBR = (1,8), CHG = +1;

IDENTIFY

	PR IN MS1+ AND
	FR in MS2+

SUCHTHAT
	(avg(FR.intensity)/avg(PR.intensity) >=0.1 AND avg(FR.intensity)/avg(PR.intensity) <=10) AND
	PR.chemsc[C]/PR.chemsc[db] >= 4.6

REPORT 

	SPECIE = "CE %d:%d" % "((PR.chemsc)[C] - 27, (PR.chemsc)[db] - 4.5)";  
	CLASS = "CE";
	MASS = PR.mass;
	ISOBARIC = PR.isobaric;
	CHEMSC = PR.chemsc;
	ERRppm = "%2.2f" % "(PR.errppm)";
	FRERRppm = "%2.2f" % "(FR.errppm)";
	INT = FR.intensity;
	QUALA = PR.intensity;
```

In [13]:
from lx2.targets import MFQL_util
from lx2.MS_reader import SpectraUtil

In [14]:
QUERYNAME = 'CholesterolMSMS'

In [15]:
elements = {'C':(41,49),'H':(30,200),'O':(2,2),'N':(1,1),'P':(1,1) }      
PR = MFQL_util(elements)
PR.set_dbr(4.5,10.5)

In [16]:
elements = {'C':(27,27),'H':(45,45) }      
FR = MFQL_util(elements)
FR.set_dbr(1,8)

In [17]:
FR._df

,C,H,m,dbr,chem
0,27,45,369.352126,5.5,C27 H45


In [18]:
filename = 'test_resources\\small_test\\190321_Serum_Lipidextract_368723_01.mzML'
spectraUtil = SpectraUtil.fromFile(filename, test_sample=True)

In [19]:
MS1 = spectraUtil
MS2 = spectraUtil.get_reset_copy()
MS1.set_mode()
MS1.set_ms_level()
MS2.set_mode()
MS2.set_ms_level(2)


a copy of the original with nothing set... sorry no undo
set mode to positive : True, (false means negative) 
set ms level to  : 1
set mode to positive : True, (false means negative) 
set ms level to  : 2


find the nearest Peaks to the target_peaks with a tolerance of 0.01


In [20]:
MS1_match = MS1.get_nearest(PR._df)
MS2_match = MS2.get_nearest(FR._df)

find the nearest Peaks to the target_peaks with a tolerance of 0.01
find the nearest Peaks to the target_peaks with a tolerance of 0.01


## get only the nearest peak in each scan

In [21]:
MS1_match.shape

(302, 12)

In [22]:
MS1_match = MFQL_util.removeDoubleID(MS1_match)
MS2_match = MFQL_util.removeDoubleID(MS2_match)

C:\Users\mirandaa\Anaconda3\envs\lx128_p3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
MS1_match.shape

(296, 13)

In [19]:
all_match = MFQL_util.makeAllCombo(MS1_match, MS2_match)

In [20]:
all_match.columns

Index(['id_x', 'm_x', 'i_x', 'index_x', 'C_x', 'H_x', 'O', 'N', 'P', 'dbr_x',
       'chem_x', 'target_x', 'err_x', 'id_y', 'm_y', 'i_y', 'index_y', 'C_y',
       'H_y', 'dbr_y', 'chem_y', 'target_y', 'err_y'],
      dtype='object')

In [21]:
# # (avg(FR.intensity)/avg(PR.intensity) >=0.1 AND avg(FR.intensity)/avg(PR.intensity) <=10) AND
# 	PR.chemsc[C]/PR.chemsc[db] >= 4.6

ST = all_match.query('FR_i/PR_i >= 0.1 and FR_i/PR_i <= 10 and PR_C/PR_dbr >= 4.6')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\mirandaa\Anaconda3\envs\lx128_p3\lib\site-packages\pandas\core\computation\scope.py", line 188, in resolve
    return self.resolvers[key]
  File "C:\Users\mirandaa\Anaconda3\envs\lx128_p3\lib\collections\__init__.py", line 916, in __getitem__
    return self.__missing__(key)            # support subclasses that define __missing__
  File "C:\Users\mirandaa\Anaconda3\envs\lx128_p3\lib\collections\__init__.py", line 908, in __missing__
    raise KeyError(key)
KeyError: 'FR_i'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mirandaa\Anaconda3\envs\lx128_p3\lib\site-packages\pandas\core\computation\scope.py", line 199, in resolve
    return self.temps[key]
KeyError: 'FR_i'

During handling of the above exception, another exception occurred:

Traceback (most recen

TypeError: can only concatenate str (not "list") to str

# make it a summary like the current lx, 
# one entry one line


In [0]:
ST[ST.PR_chem== 'C43 H71 O2 N1 P1 ']

In [0]:
#clean up and only keep the closest peak per scan

In [0]:
for pe,pg in ST.groupby('PR_chem'):
    pg = pg.round(4)
    print(f'PR: {pe}: {pg.PR_m.mean():.4f} {pg.PR_m.std():.4f}, {pg.PR_i.mean():.4f} {pg.PR_i.std():.4f}, {pg.PR_m.count()}')
    for fe, fg in pg.groupby('FR_chem'):
        fg = fg.round(4)
        print(f'FR: {fe}: {fg.FR_m.mean():.4f} {fg.FR_m.std():.4f}, {fg.FR_i.mean():.4f} {fg.FR_i.std():.4f}, {fg.FR_m.count()}')
    print('----------')
    